### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [2]:
from matplotlib import pyplot as plt
import os
import numpy as np
import cv2 # OpenCV
from calibration import Calibrator

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [3]:
calibrator = Calibrator()

folderpath = './images'
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()
limages = []

for path in images_path:
    image = cv2.imread(path)
    limages.append(image)


calibrator.fit(limages, radial_distortion=True)

print(calibrator.reprojection_errors)
print(calibrator.K)

print(calibrator.distortion_parameters)

True
eseguito
[np.float64(3.3913191982671673), np.float64(3.0465297392724278), np.float64(26.7397359536844), np.float64(1.4687380109654402), np.float64(2.0926707585066806), np.float64(48.381874928154495), np.float64(9.83780286370121), np.float64(261.8423224784871), np.float64(13.974516320955427), np.float64(9.763160933292456), np.float64(6.933401866214631), np.float64(10.033768168021092), np.float64(7.347656839601976), np.float64(19.67950400520422), np.float64(19.915813215048367), np.float64(12.42458749893394), np.float64(53.66131107842198), np.float64(52.27735212586703), np.float64(11.108996191298717), np.float64(20.751577048909994)]
[[1.74422018e+03 1.04745713e+00 6.32237773e+02]
 [0.00000000e+00 1.74503696e+03 4.97985133e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.00381553]
 [ 0.10023698]]


6. superimposition di un oggetto per tutte le immagini

In [4]:
pts = cv2.ellipse2Poly((44,55), (22,22), 0, 0, 360, 1)
pts = pts/1000
et_low = np.empty_like(pts)
et_high = np.empty_like(et_low)
pts_low = np.hstack((pts, np.zeros((pts.shape[0],1)), np.ones((pts.shape[0],1))))
pts_high = np.hstack((pts, np.full((pts.shape[0],1), 0.055), np.ones((pts.shape[0],1))))

In [ ]:
for i,path in enumerate(images_path):
    original = cv2.imread(path)
    H, R, I = calibrator.estimate_homography(original)
    P = calibrator.estimate_projection_matrix(H, calibrator.K)
    error = calibrator.reprojection_error(P, R, I)
    proj_chess = np.transpose(P@R.T)
    proj_chess[:,0] = proj_chess[:,0]/proj_chess[:,2]
    proj_chess[:,1] = proj_chess[:,1]/proj_chess[:,2]
    proj_chess = proj_chess[:,:-1]
    img1 = original.copy()
    for point in proj_chess:
        img1 = cv2.drawMarker(img1, tuple(np.array(point, dtype=np.int32)), (255,0,0), markerType=cv2.MARKER_STAR)
    
    proj_low = np.transpose(P @ pts_low.T)
    et_low[:,0] = proj_low[:,0]/proj_low[:,2]
    et_low[:,1] = proj_low[:,1]/proj_low[:,2]
    proj_high = np.transpose(P @ pts_high.T)
    et_high[:,0] = proj_high[:,0]/proj_high[:,2]
    et_high[:,1] = proj_high[:,1]/proj_high[:,2]
    #print(proj_high)
    img2 = original.copy()
    img2 = cv2.fillConvexPoly(img2,np.array(et_low, dtype=np.int32),(255,0,0))
    img3 = cv2.fillConvexPoly(img2.copy(),np.array(et_high, dtype=np.int32),(0,255,0))
    img4 = original.copy()
    for i in range(et_low.shape[0]):
        img4 = cv2.line(img4, tuple(np.array(et_low[i], dtype=np.int32)), tuple(np.array(et_high[i], dtype=np.int32)),(0,0,255))
    img2 = cv2.addWeighted(img2, 0.5, original, 0.5, 0)
    img3 = cv2.addWeighted(img2, 0.5, img3, 0.5, 0)
    img4 = cv2.addWeighted(img3, 0.5, img4, 0.5, 0)
    plt.figure(figsize=(20,20))
    plt.imshow(img1)
    plt.title(f"Error: {error}")
    plt.figure(figsize=(20,20))
    plt.imshow(img4)
    plt.title(f"{proj_high[0,2]}")